# Optimization with a Random Forest Surrogate Model on Total reading time

- Model: Random Forest, linear interpolation for sd
- Modelled variables: total reading time
- Optimized variables: total reading time, Sharpe ratio of total reading time

In [23]:
import os
import sys
from functools import partial

import joblib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import minimize_scalar
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from smt.surrogate_models import KRG
from scipy import interpolate

sys.path.append("../src")
from preprocessing import prepare_data
from plot_estimated_reading import plot_reading_for_2d_allocation, plot_optimization_improvement
from estimate_reading import reading_function, compute_reading_for_proportion


## Prepare data for modeling

In [24]:
df_raw = pd.read_csv("../data/raw_reading_samples.csv")
df = prepare_data(df_raw, format_to_long=True)
df.head(5)

,reading_fiction,reading_help,free_fiction,free_help,total_reading
0,9.342691,9.983322,0.0,0.0,19.326013
1,9.515911,11.093644,0.0,0.0,20.609556
2,11.628954,7.527137,0.0,0.0,19.156091
3,9.763141,10.031910,0.0,0.0,19.795051
4,9.741243,9.705921,0.0,0.0,19.447164


## Modeling

In [25]:
X = df[['free_fiction', 'free_help']].values
Y = df['total_reading'].values

In [26]:
# training the surrogate model for total reading time directly
forest = RandomForestRegressor(n_estimators=10, random_state=42)
%time forest.fit(X, Y)

CPU times: user 97.1 ms, sys: 3.75 ms, total: 101 ms
Wall time: 99.5 ms


RandomForestRegressor(n_estimators=10, random_state=42)

In [27]:
joblib.dump(forest, "../models/RandomForest.joblib")

['../models/RandomForest.joblib']

In [28]:
# Linear interpolation to estimate sd
df_sd = df[['free_fiction', 'free_help', 'total_reading']].groupby(['free_fiction', 'free_help']).std().reset_index()
x, y, z = df_sd.free_fiction, df_sd.free_help, df_sd.total_reading
std_interpolator = interpolate.interp2d(x, y, z, kind='linear')
std_interpolator(0, 20)

array([1.02334715])

In [29]:
#predictions
Y_predict = forest.predict(X)

In [30]:
# Evaluate model
r2_score(Y, Y_predict)

0.99930782968815

## Plot reading function
### 1. Sample predictions over allocations of the whole time budget

In [31]:
# Sample space and create budget allocations all over it
total_time = 120
prop_fiction = np.arange(start=0, stop=1.01, step=0.01)
allocations = np.array(np.array([prop_fiction, (1 - prop_fiction)]) * total_time).transpose()

In [32]:
# Predict reading and its standard deviation
expected_reading = forest.predict(allocations)
sd_reading = np.array([std_interpolator(a[0], a[1])[0] for a in zip(allocations[:, 0], allocations[:, 1])])
sharpe_reading = expected_reading / sd_reading

In [33]:
sd_reading.shape

(101,)

In [34]:
expected_reading.shape

(101,)

In [35]:
# Summarize predictions
df_predictions = pd.DataFrame()
df_predictions['prop_fiction'] = prop_fiction
df_predictions['Expected reading'] = expected_reading
df_predictions['Std reading'] = sd_reading
df_predictions['Sharpe reading'] = sharpe_reading

In [36]:
# Plot sales for various proportions between English and Spanish
fig = px.scatter(df_predictions, x="prop_fiction", y="Expected reading")
fig.update_layout(
    title="Estimatation of expected reading for all allocations of 120 minutes",
    title_x=0.5,
    xaxis_title="Proportion of the free time for Fiction",
    yaxis_title="Expected reading",
)
fig.show()

In [37]:
fig = px.scatter(df_predictions, x="prop_fiction", y="Std reading")
fig.update_layout(
    title="Predicted std of reading for all allocations of 120 minutes",
    title_x=0.5,
    xaxis_title="Proportion of the free time for Fiction",
    yaxis_title="Std reading",
)
fig.show()

In [38]:
# Plot sales for various proportions between English and Spanish
fig = px.scatter(df_predictions, x="prop_fiction", y="Sharpe reading")
fig.update_layout(
    title="Predicted Sharpe ratio of reading time for all allocations of 120 minutes",
    title_x=0.5,
    xaxis_title="Proportion of the free time for Fiction",
    yaxis_title="Sharpe ratio of reading time",
)
fig.show()

### 3. Sample all allocations of free time below 120 minutes

In [39]:
plot_reading_for_2d_allocation(
    max_time=120, 
    estimators=forest, 
    type_estimator="sklearn", 
    sampling_step=10,
)

In [40]:
plot_reading_for_2d_allocation(
    max_time=120, 
    estimators=forest, 
    type_estimator="sklearn", 
    sampling_step=10,
    estimator_sd=std_interpolator,
)

## Optimization
### 1. Optimize estimated expected reading function

In [41]:
fun_reading = partial(
    compute_reading_for_proportion, 
    total_budget = 120, 
    estimators=forest, 
    type_estimator="sklearn", 
    sign=-1
) 

In [42]:
res = minimize_scalar(fun_reading, bounds=(0, 1), method='bounded')
print("Found an optimum when p = {:,.3f}, then total reading time is {:,.0f} min".format(res.x[0], - res.fun[0]))

Found an optimum when p = 0.236, then total reading time is 204 min


### 2. Optimize Sharpe ratio of reading time

In [43]:
fun_reading_sharpe = partial(
    compute_reading_for_proportion, 
    total_budget = 120, 
    estimators=forest, 
    type_estimator="sklearn", 
    estimator_sd=std_interpolator,
    sign=-1
) 

In [44]:
res = minimize_scalar(fun_reading_sharpe, bounds=(0, 1), method='bounded')
print("Found an optimum when p = {:,.3f}, then Sharpe ratio of reading time is {:,.0f}".format(res.x[0], - res.fun[0]))

Found an optimum when p = 0.500, then Sharpe ratio of reading time is 97
